# Criação de Database e tabelas no BigQuery no GCP Usando Python

## Libs

In [1]:
import os
import logging
from typing import List, Dict, Any, Optional
import json  

from google.cloud import bigquery
from google.api_core import exceptions

## Credenciais

In [ ]:
def setup_bigquery_credentials(
    credentials_path: Optional[str] = None, 
    credentials_env_var: str = "GOOGLE_APPLICATION_CREDENTIALS",
    verbose: bool = True
) -> bigquery.Client:
    """
    Configura as credenciais do BigQuery e inicializa o cliente. 

    Args:
        credentials_path (str, opcional): Caminho para o arquivo de credenciais JSON
        credentials_env_var (str, opcional): Nome da variável de ambiente para credenciais
        verbose (bool, opcional): Exibir logs detalhados
    
    Returns:
        bigquery.Client: Cliente do BigQuery
    """
    # Configurar logging 
    logging.basicConfig(
        level=logging.INFO if verbose else logging.WARNING,
        format='%(asctime)s - %(levelname)s: %(message)s'
    )
    logger = logging.getLogger(__name__)

    try:
        # Se o caminho não for fornecido, tenta usar localizações padrão 
        if credentials_path is None:
            default_paths = [
                "../key.json",
                "../../key.json",
                os.path.expanduser("~/key.json")
            ]
            
            for path in default_paths:
                if os.path.exists(path):
                    credentials_path = path
                    break

        # Validar existência do arquivo de credenciais 
        if not credentials_path or not os.path.exists(credentials_path):
            raise FileNotFoundError(f"Arquivo de credenciais não encontrado em {credentials_path}")

        # Configurar variável de ambiente 
        os.environ[credentials_env_var] = credentials_path
        logger.info(f" 🔑 Credenciais configuradas: {credentials_path}")

        # Inicializar cliente do BigQuery 
        bqclient = bigquery.Client()
        
        return bqclient

    except Exception as e:
        logger.error(f"Erro ao configurar credenciais: {e}")
        raise

## Criação de Dataset

In [ ]:
def create_bigquery_dataset(
    client: bigquery.Client, 
    dataset_id: str, 
    location: str = "US",
    verbose: bool = True
) -> bigquery.Dataset:
    """
    Cria um dataset no BigQuery, verificando se já existe. 

    Args:
        client (bigquery.Client): Cliente do BigQuery
        dataset_id (str): ID completo do dataset (projeto.dataset)
        location (str, opcional): Localização geográfica do dataset
        verbose (bool, opcional): Exibir logs detalhados

    Returns:
        bigquery.Dataset: Dataset criado ou existente
    """
    # Configurar logging 
    logging.basicConfig(
        level=logging.INFO if verbose else logging.WARNING,
        format='%(asctime)s - %(levelname)s: %(message)s'
    )
    logger = logging.getLogger(__name__)

    try:
        # Construir objeto do Dataset 
        dataset = bigquery.Dataset(dataset_id)
        dataset.location = location

        try:
            # Tentar criar dataset, capturando se já existir 
            dataset = client.create_dataset(dataset, timeout=30)
            logger.info(f"✅ Dataset criado: {dataset_id}")
        except exceptions.Conflict:
            logger.warning(f"⚠️ Dataset {dataset_id} já existe. Usando o existente.")
            dataset = client.get_dataset(dataset_id)

        return dataset

    except Exception as e:
        logger.error(f"❌ Erro ao criar dataset: {e}")
        raise

## Criação de Tabelas

In [ ]:
def create_bigquery_tables(
    client: bigquery.Client, 
    dataset_id: str, 
    schema_path: str = 'schema.json',
    verbose: bool = True
) -> Dict[str, bigquery.Table]:
    """
    Cria tabelas no BigQuery a partir de um arquivo de schema JSON. 

    Args:
        client (bigquery.Client): Cliente do BigQuery
        dataset_id (str): ID do dataset onde as tabelas serão criadas
        schema_path (str): Caminho para o arquivo de schema JSON
        verbose (bool, opcional): Exibir logs detalhados

    Returns:
        Dict[str, bigquery.Table]: Dicionário de tabelas criadas
    """
    # Configurar logging 
    logging.basicConfig(
        level=logging.INFO if verbose else logging.WARNING,
        format='%(asctime)s - %(levelname)s: %(message)s'
    )
    logger = logging.getLogger(__name__)

    try:
        # Verificar existência do arquivo de schema 
        if not os.path.exists(schema_path):
            raise FileNotFoundError(f"Arquivo de schema não encontrado em: {schema_path}")

        # Carregar schema do arquivo JSON 
        with open(schema_path, 'r') as schema_file:
            schemas = json.load(schema_file)

        # Validar estrutura do schema 
        if not schemas or not isinstance(schemas, dict):
            raise ValueError("Schema inválido. Deve ser um dicionário não vazio.")

        created_tables = {}

        # Iterar sobre as definições de tabela no schema 
        for table_name, table_info in schemas.items():
            # Validar informações da tabela 
            if 'schema' not in table_info:
                logger.warning(f"⚠️ Pulando tabela {table_name}: Schema não definido")
                continue

            # Construir ID completo da tabela 
            full_table_id = f"{dataset_id}.{table_name}"
            
            # Criar objeto de tabela 
            table = bigquery.Table(full_table_id, schema=table_info['schema'])
            table.description = table_info.get('description', '')

            try:
                # Tentar criar tabela 
                created_table = client.create_table(table)
                logger.info(f"✅ Tabela criada: {full_table_id}")
                created_tables[table_name] = created_table
            except exceptions.Conflict:
                logger.warning(f"⚠️ Tabela {full_table_id} já existe. Pulando...")
            except Exception as table_error:
                logger.error(f"❌ Erro ao criar tabela {full_table_id}: {table_error}")

        if not created_tables:
            logger.warning("⚠️ Nenhuma tabela foi criada")

        return created_tables

    except json.JSONDecodeError:
        logger.error("❌ Erro ao decodificar o arquivo JSON de schema")
        raise
    except Exception as e:
        logger.error(f"❌ Erro ao criar tabelas: {e}")
        raise

In [ ]:
if __name__ == "__main__":
    try:
        # Configurar credenciais e cliente
        bq_client = setup_bigquery_credentials()

        # Criar dataset 
        dataset_id = f"{bq_client.project}.db_retail"
        dataset = create_bigquery_dataset(bq_client, dataset_id)

        # Criar tabelas 
        tables = create_bigquery_tables(bq_client, dataset_id)

    except Exception as e:
        print(f"Erro no processo: {e}")

2024-12-15 15:29:32,067 - INFO: Credenciais configuradas: ../../key.json
2024-12-15 15:29:33,386 - INFO: ✅ Dataset criado: bankmarketingdatapipeline.db_retail
2024-12-15 15:29:33,797 - INFO: ✅ Tabela criada: bankmarketingdatapipeline.db_retail.trusted_departments
2024-12-15 15:29:34,288 - INFO: ✅ Tabela criada: bankmarketingdatapipeline.db_retail.trusted_order
2024-12-15 15:29:34,872 - INFO: ✅ Tabela criada: bankmarketingdatapipeline.db_retail.trusted_categories
2024-12-15 15:29:35,383 - INFO: ✅ Tabela criada: bankmarketingdatapipeline.db_retail.trusted_products
2024-12-15 15:29:35,793 - INFO: ✅ Tabela criada: bankmarketingdatapipeline.db_retail.trusted_customers
2024-12-15 15:29:36,179 - INFO: ✅ Tabela criada: bankmarketingdatapipeline.db_retail.trusted_order_items
